In [26]:
import sys
import os
# add parent directory (where "natsume" lives) to sys.path so I can import natsume
sys.path.append(r"C:\Users\WBS\Desktop\EXOPLANET WORK\006 NATSUME")
import natsume

import ttv_curvefit.curvefit as ttvfit  # custom-written
import ttvfast
from ttvfast import models

from tqdm import tqdm
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.constants import M_earth
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

In [27]:
# Create 5000 samples for TTVFast for each MMR
size = 5000 * 10
mu1 = 3e-6; mu2 = 3e-6
e1 = 10**np.random.uniform(low=-3, high=-1, size=size); e2 = e1.copy()
w1 = np.random.uniform(low=0, high=360, size=size)
w2 = (w1.copy() + np.random.choice([0, 180], size=size)) % 360
Delta = np.random.uniform(low=-0.02, high=0.02, size=size)

mmr = np.random.choice(['2:1', '3:2', '4:3', '5:4', '3:1', '5:3', '7:5', '4:1', '5:2', '5:1'],
                       size=size)
mmrparts = np.char.split(mmr, ':')  # Extract j:j-N
j = np.array([int(p[0]) for p in mmrparts])
N = np.array([int(p[0]) - int(p[1]) for p in mmrparts])

P1 = 30; P2 = P1 * (Delta + 1) * j / (j-N)

# Compile to dataframe
df = pd.DataFrame({
    'e1': e1, 'w1': w1,
    'e2': e2, 'w2': w2,
    'Delta': Delta,
    'mmr': mmr, 'j': j, 'N': N,
    'P1': P1, 'P2': P2,
    'mu1': mu1, 'mu2': mu2,
    'valid': True
})
df.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,mu1,mu2,valid
0,0.015016,135.786278,0.015016,315.786278,-0.018080,3:2,3,1,30,44.186407,0.000003,0.000003,True
1,0.011390,214.359496,0.011390,214.359496,-0.001431,7:5,7,2,30,41.939893,0.000003,0.000003,True
2,0.001320,41.057194,0.001320,221.057194,-0.012968,5:3,5,2,30,49.351578,0.000003,0.000003,True
3,0.015594,22.256611,0.015594,202.256611,0.014695,5:1,5,4,30,152.204196,0.000003,0.000003,True
4,0.020713,348.970550,0.020713,168.970550,0.018677,5:4,5,1,30,38.200388,0.000003,0.000003,True


In [28]:
# Copy dataframe for fitting results
df_fit = df.copy()
df_fit_cols = ["A1", "B1", "V1", "Pttv1", "phase1", "std1", "A2", "B2", "V2", "Pttv2", "phase2", "std2", "R2_1", "R2_2"]
df_fit[df_fit_cols] = np.nan
df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,phase1,std1,A2,B2,V2,Pttv2,phase2,std2,R2_1,R2_2
0,0.015016,135.786278,0.015016,315.786278,-0.018080,3:2,3,1,30,44.186407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.011390,214.359496,0.011390,214.359496,-0.001431,7:5,7,2,30,41.939893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.001320,41.057194,0.001320,221.057194,-0.012968,5:3,5,2,30,49.351578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.015594,22.256611,0.015594,202.256611,0.014695,5:1,5,4,30,152.204196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.020713,348.970550,0.020713,168.970550,0.018677,5:4,5,1,30,38.200388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# TTVFast simulation (Copypasted from 1st order code)
# Create TTV simulations and extract Amplitude and Period
gravity = 0.000295994511 # AU^3/day^2/M_sun
stellar_mass = 1.0       # M_sun

for i, row in tqdm(df.iterrows(), total=len(df)):  
    # Initialize variables
    P1 = row['P1']; P2 = row['P2']
    j = row['j']; N = row['N']
    PTTV = 1/np.abs(j/P2 - (j-N)/P1)
    
    e1 = row['e1']; w1 = row['w1']
    e2 = row['e2']; w2 = row['w2']
    mu1 = row['mu1']; mu2 = row['mu2']
    
    # Create TTVFast planets
    planet1 = models.Planet(
        mass=mu1,            # M_sun
        period=P1,           # days
        eccentricity=e1,
        inclination=90,      # degrees
        longnode=0,          # degrees
        argument=90+w1,      # degrees
        mean_anomaly=0       # degrees
    )
    planet2 = models.Planet(
        mass=mu2,
        period=P2,
        eccentricity=e2,
        inclination=90,
        longnode=0,
        argument=90+w2,
        mean_anomaly=0,
    )
    planets = [planet1, planet2]
    
    begin_time = -P1 * 50
    Time = begin_time  # days
    dt = P1 / 100       # days
    Total = min(1600, PTTV * 2.5)   # days
    
    results = ttvfast.ttvfast(planets, stellar_mass, Time, dt, Total,
                              input_flag=1)

    # Create TTV signal from TTVFast Output
    out = pd.DataFrame(results['positions']).transpose()
    out.columns = ['planet', 'epoch', 'times', 'rsky', 'vsky']
    
    table0 = out[(out.planet == 0) & (out.times > 0)]
    n0 = table0.epoch
    tn0 = table0.times

    table1 = out[(out.planet == 1) & (out.times > 0)]
    n1 = table1.epoch
    tn1 = table1.times

    # If few TTVFast output, skip
    len_threshold = 10
    if (len(n0) <= len_threshold) or (len(n1) <= len_threshold):
        continue
        
    time0, ttv0, ttv0_err = ttvfit.return_ttv(epochs=n0, midtransits=tn0)
    time1, ttv1, ttv1_err = ttvfit.return_ttv(epochs=n1, midtransits=tn1)
    
    # Make sure ttv0 and ttv1 actually are centered around O-C = 0
    ttv0 = ttv0 - (np.max(ttv0) + np.min(ttv0)) / 2
    ttv1 = ttv1 - (np.max(ttv1) + np.min(ttv1)) / 2
    
    # Get sinusoidal TTV model parameters
    popt_ttv0, perr_ttv0, r2_0 = ttvfit.characterize_ttv(time=time0, ttv=ttv0, ttv_err=ttv0_err,
                                                         expected_period=PTTV, method='lmfit')
    popt_ttv1, perr_ttv1, r2_1 = ttvfit.characterize_ttv(time=time1, ttv=ttv1, ttv_err=ttv1_err,
                                                         expected_period=PTTV, method='lmfit')
    std0, std1 = np.std(ttv0), np.std(ttv1)

    # Write results to df_fit
    df_fit.loc[i, df_fit_cols]  = [*popt_ttv0, std0, *popt_ttv1, std1, r2_0, r2_1]

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [1:27:34<00:00,  9.52it/s]


In [30]:
# If output fit has no values, convert valid from TRUE to FALSE
output_has_nan = df_fit[df_fit_cols].isna().any(axis=1)
df_fit.loc[output_has_nan, 'valid'] = False

# If output has PTTV wildly different from each other, convert valid from TRUE to FALSE
Pttv_ratio_threshold = 1.05
Pttv_not_similar = (df_fit['Pttv1'] / df_fit['Pttv2'] > Pttv_ratio_threshold) | (df_fit['Pttv2'] / df_fit['Pttv1'] > Pttv_ratio_threshold)
df_fit.loc[Pttv_not_similar, 'valid'] = False

df_fit.head()

,e1,w1,e2,w2,Delta,mmr,j,N,P1,P2,...,phase1,std1,A2,B2,V2,Pttv2,phase2,std2,R2_1,R2_2
0,0.015016,135.786278,0.015016,315.786278,-0.018080,3:2,3,1,30,44.186407,...,1.966894,0.003061,-0.000834,1.021793e-06,0.006055,812.094692,5.079151,0.004280,0.988654,0.993372
1,0.011390,214.359496,0.011390,214.359496,-0.001431,7:5,7,2,30,41.939893,...,8.997557,0.000518,0.016647,-2.083234e-05,0.063693,19078.532890,6.020330,0.000681,0.000235,0.001789
2,0.001320,41.057194,0.001320,221.057194,-0.012968,5:3,5,2,30,49.351578,...,4.570373,0.000226,-0.000011,7.036167e-09,0.000062,785.713741,7.439063,0.000227,0.013201,0.037107
3,0.015594,22.256611,0.015594,202.256611,0.014695,5:1,5,4,30,152.204196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.020713,348.970550,0.020713,168.970550,0.018677,5:4,5,1,30,38.200388,...,3.640849,0.004922,-0.001175,1.860154e-06,0.005087,388.548663,0.792823,0.005726,0.376617,0.397031


In [31]:
# Export Outputs
df_fit.to_csv("allorder_TTVFast_fits.csv", index=False)